# Day Thirteen

## Task

The [task](https://adventofcode.com/2017/day/13) today is to determine the severity of a trip through a firewall. In this case, the firewall is made up of several layers. In each layer there is a scanner that moves up and down. The idea is that we have a preset path (in part one at least) for which we must compute the severity; the number of times that we encounter the scanner.

First, let's look in more detail at one of the layers. According to the task, a layer has a range and a depth. The range indicates how many steps the scanner has to travel across the layer. For example, the following layer has a range of three.

```
[ ]
[ ]
[ ]
```

The depth of a layer indicates where in the firewall the layer is. Here, we have a firewall with two layers, one with a range of three and depth of zero, and the other with a range of two and depth of one.

```
[ ]   [ ]
[ ]   [ ]
[ ]
```

Finally, let's look at how the scanner travels through a layer. It spends one tick in each step, so letting time increase from left to right, we have

```
t=0    t=1    t=2    t=3    t=4    t=5    t=6

[S]    [ ]    [ ]    [ ]    [S]    [ ]    [ ] 
[ ] -> [S] -> [ ] -> [S] -> [ ] -> [S] -> [ ] ...
[ ]    [ ]    [S]    [ ]    [ ]    [ ]    [S] 
```

### Part One

First, let's define a funtion that can give us the position of a scanner in a layer with given range after some time.

In [1]:
inc = lambda x: x + 1
dec = lambda x: x - 1

    
def position(range):
    p = -1
    f = inc

    while True:
        p = f(p)
        
        if p == 0:
            f = inc
        
        if p + 1 == range:
            f = dec

        yield p

Here, `position` returns a generator that on iteration gives the location of the scanner.

In [2]:
i = position(3)
for t in range(0, 5):
    print(f'At t={t}, scanner is about to leave position {next(i)}')

At t=0, scanner is about to leave position 0
At t=1, scanner is about to leave position 1
At t=2, scanner is about to leave position 2
At t=3, scanner is about to leave position 1
At t=4, scanner is about to leave position 0


Now let's think about the severity of a path through the firewall. According to the task, when `t` increments the following happens:

 - We move one step to the right.
 - If there is a scanner in the place we've just moved into, we're caught.
 - Then the scanners move one step in whatever direction they're travelling.
 
Using the example given in the task, we write this as follows:

In [3]:
def severity(firewall):
    sev = 0
    
    positions = [position(x) if x else None for x in firewall]
    
    for t in range(0, len(positions)):
        positions_at_t = [next(x) if x else None for x in positions]
        
        if positions_at_t[t] == 0:
            sev += t * firewall[t]
            
    return sev
            
            
severity([3, 2, None, None, 4, None, 4])

24

Using the puzzle data, this becomes

In [4]:
def read():
    with open('../../data/day13.txt') as f:
        data = f.read()
        
    data_dict = {}
    for x in data.split('\n'):
        if not x:
            continue
            
        k, v = map(int, x.split(': '))
        data_dict[k] = v
                
    return [data_dict.get(x, None) for x in range(0, max(data_dict.keys()) + 1)]


data = read()
severity(data)

1580

### Part Two

It turns out that the solution is sufficiently large that I need to use a different approach to part one (see [this commit](https://github.com/jdgillespie91/aoc/commit/26833fd78e40d00e4faa450041e83e13a35eca46) for the results when using this approach; it's very slow). In principle, the idea is to track whether a scanner is at the top of its layer or not rather than tracking _exactly_ where it is.

In [5]:
class Scanner:
    def __init__(self, range):
        self._range = range
        
    @property
    def range(self):
        return self._range
    
    def __repr__(self):
        return f'<Scanner ({self.range})>'


def is_scanner_at_top(scanner, time):
    if time % (2*scanner.range - 2) == 0:
        return True
    
    return False

Now we can define a function like `severity` again.

In [6]:
def severity(firewall):
    sev = 0
    
    for time, range_ in enumerate(firewall):
        if range_ and is_scanner_at_top(Scanner(range_), time):
            sev += time * range_
            
    return sev

Similarly, we can define something like `caught` too.

In [7]:
def caught(firewall, delay):   
    for time, range_ in enumerate(firewall, start=delay):
        if range_ and is_scanner_at_top(Scanner(range_), time):
            return True
            
    return False

Finally, we can loop over positive delays to find the first available that results in us sneaking through the firewall.

In [8]:
firewall = read()
delay = 0
while True:
    if not caught(firewall, delay=delay):
        print(f'We sneak through with a delay of {delay} picoseconds!')
        break
    
    delay += 1

We sneak through with a delay of 3943252 picoseconds!
